In [1]:
!pip install tweepy

In [2]:
!pip install wordcloud

In [3]:
!pip install plotly

In [4]:
import tweepy
import re 
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns 
import plotly.express as px

In [26]:
#insert you bearer in a text file called bearer.txt
with open("bearer.txt", "r") as f:
    MY_BEARER_TOKEN = str(f.readlines()[0])
client = tweepy.Client(bearer_token=MY_BEARER_TOKEN)

In [27]:
# query to search for tweets
query = "Covid lang:en"

# your start and end time for fetching tweets
start_time = "2022-07-23T00:00:00Z"
end_time = "2022-07-28T00:00:00Z"

# get tweets from the API
tweets = client.search_recent_tweets(query=query,
                                     start_time=start_time,
                                     end_time=end_time,
                                     tweet_fields = ["created_at", "text", "source"],
                                     user_fields = ["name", "username", "location", "verified", "description"],
                                     max_results = 100,
                                     expansions='author_id'
                                     )

In [ ]:
# import the pandas library
import pandas as pd

# create a list of records
tweet_info_ls = []

# iterate over each tweet and corresponding user details
for tweet, user in zip(tweets.data, tweets.includes['users']):
    tweet_info = {
        'created_at': tweet.created_at,
        'text': tweet.text,
        'source': tweet.source,
        'name': user.name,
        'username': user.username,
        'location': user.location,
        'verified': user.verified,
        'description': user.description
    }
    tweet_info_ls.append(tweet_info)

# create dataframe from the extracted records
tweets_df = pd.DataFrame(tweet_info_ls)
# display the dataframe
tweets_df

In [ ]:
pip install textblob

In [ ]:
tweets1 = tweets_df.iloc[:,1:2]
tweets1

In [ ]:
for i in range(0,len(tweets1)):
    print(tweets1.iloc[i])

In [ ]:
# Let's create a function that clean the tweets
def clean(text):
    text = re.sub(r'@[A-Za-z0-9]+',' ',text) #remove @mentions
    text = re.sub(r'#','',text) #remove #
    text = re.sub(r'RT[\s]+','',text) #remove RT
    text = re.sub(r'https?:\/\/\S+','',text)
    
    return text

In [ ]:
tweets1['text'] = tweets1['text'].apply(clean)
tweets1

In [ ]:
#create a function to get the subjectivity
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity
#create a function to get the polarity
def getpolarity(text):
    return TextBlob(text).sentiment.polarity


In [ ]:
df = tweets1.copy()
df['subjectivity'] = df['text'].apply(getsubjectivity)
df['polarity'] = df['text'].apply(getpolarity)
df

In [ ]:
# allwords put together all the tweets 
allwords = ' '.join([twts for twts in df['text']])
wordCloud = WordCloud(width=500,height=300,random_state=22,max_font_size=119,collocations=False).generate(allwords)
plt.figure(figsize = (8,20))
plt.imshow(wordCloud,interpolation='nearest')
plt.axis('off')
plt.show()

In [ ]:
def getanalisys(score):
    if score < 0 :
        return 'Negative'
    if score == 0:
        return 'Neutral'
    if score > 0:
        return 'Positive'
        

In [ ]:
df['Analysis'] = df['polarity'].apply(getanalisys)
df

In [ ]:
fig = px.histogram(df, x="Analysis")
fig.show()